In [40]:
print('Started...')
import glob
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import png
import time
import numpy as np
import os
from fnmatch import fnmatch


# Gobals
CROP_SIZE = 256
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/malignant/' #1.2.840.113681.2230565232.95*'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/normal/'#1.2.840.113681.2230565232.95*'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/1/optm10'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/1'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/15'


# File walk method
def getFiles(verbose=1):
    fileList = []
    for path, subdirs, files in os.walk(DICOM_FILES):
        for name in files:
            if fnmatch(name, '*.dcm'):
                fileList.append(os.path.join(path, name))

    print(len(fileList), ' dicom images found')


    # Load dicom files into np array
    dicomImg = np.array([])
    count = 0
    print(len(fileList), ' Files found')
    print('Loading images...')
    for f in fileList:
        dicomImg = np.append(dicomImg, pydicom.dcmread(f))
        count += 1
        print(count, '/', len(fileList))
    return dicomImg, fileList



    print('name:\n', dicomImg[0].PresentationIntentType)

def deletePreProcessed(dicomImg, fileList):
    forPresentationCount = 0
    toDelete = []
    for index, _ in enumerate(dicomImg):
        tmp = _.PresentationIntentType
        if tmp == 'FOR PRESENTATION':
            forPresentationCount += 1
        else:
            toDelete.append(index)
    print('dicomImg len:', len(dicomImg))
    np.delete(dicomImg, toDelete)
    mask = np.ones(len(dicomImg), dtype=bool)
    mask[toDelete] = False
    dicomImg = dicomImg[mask]
    fileList = np.asarray(fileList)
    fileList = fileList[mask]
    print('\nTotal DICOM: ', len(dicomImg))
    print('Total FOR PROCESSING: ', forPresentationCount)
    return dicomImg, fileList

   
# Import xls file, extract ROI coords, get pixel array from DICOM image
def buildDict(dicomImg, fileList):
    #xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/download/batch_1_IMAGE.xls')
    xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch_50_IMAGE.xls')
    #xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/normals.xls')
    sheet = xls.parse(1)

    # Create a dict where the key is the image name
    # Each key has the image, and coords
    img = {}
    for i in range(len(fileList)):
        key = os.path.basename(fileList[i])[:-4]
        # Find row in the xml file that holds the img info
        try:
            indx = [_==key for _ in sheet['ReferencedSOPInstanceUID']].index(True) # ImageSOPIUID, ReferencedSOPInstanceUID
            img.update({key:{}})
            img[key].update({'img': dicomImg[i].pixel_array})
            img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
            img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})
        except ValueError:
            print('MY_ERROR: key not found:\n', key)

    print(len(img), 'DICOM images extracted')
    # print(img)
    # Crop the images to given ROI
    toDelete = [] # Keep track of error causing keys and delete after loop
    for key in img:
        try:
            tmp = img[key]['img']
            x = img[key]['x']
            y = img[key]['y']
            x = [int(x[0]), int(x[1])]
            y = [int(y[0]), int(y[1])]
            #x is width, y is height
            #in numpy array, y,x
            img[key].update({'cropROI':tmp[y[0]:y[1], x[0]:x[1]]}) 
        except:
            print('ROI extraction failed...Removing key\nkey:  ',key, '\nx  :', x,'\ny:  ',y)
            toDelete.append(key)

    #img_calc = img['1.2.840.113681.2230565232.954.3504500766.32'] 
    for _ in toDelete:
        img.pop(_)
    return img


# Write images to disk with markers and basic crop    
def writeMarkedImages(img): 
    for key in img:
        plt.figure(figsize=(20,20))
        marker = [(img[key]['x'][1] + img[key]['x'][0])/2, (img[key]['y'][1] + img[key]['y'][0])/2 ]   
        plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
        plt.plot(marker[0], marker[1], marker='x', color=[1,0,1], markersize=30)
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_full.png')
        #plt.show()
        plt.close()

        plt.figure(figsize=(20,20))
        plt.imshow(img[key]['cropROI'], cmap='gray') 
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_crop.png')   
        #plt.show()
        plt.close()

# Crop the images so that the ROI is centred but all crops are the same size
def computeCrops(img):
    for key in img:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        c = [round((x[0]+x[1])/2), round((y[0]+y[1])/2)]
        # Pad images before cropping (wrap around)
        pad = 1000
        tmp = np.pad(tmp, pad, mode='wrap')
        img[key].update({'crop': tmp[int(c[1]-CROP_SIZE/2+pad):int(c[1]+CROP_SIZE/2+pad), int(c[0]-CROP_SIZE/2+pad):int(c[0]+CROP_SIZE/2+pad)]})
        # Reshape from (256, 256) to (256, 256, 1)
        img[key]['crop'] = np.reshape(img[key]['crop'],(img[key]['crop'].shape[0], img[key]['crop'].shape[1], 1))
    return img
    
# Find bit depth
def findBitDepth(img):
    print('Find bit depth...')
    maxmax = 0
    for key in img:
        tmp = img[key]['img']
        print(np.amax(tmp))
        if np.amax(tmp) > maxmax:
            maxmax = np.amax(tmp)
    print('The largest value is: ', maxmax)
    
# Find average ROI size
def findAverageROISize(img):
    print('Find average ROI size...')
    totalX = 0
    totalY = 0
    for key in img:
        x = img[key]['x']
        y = img[key]['y']
        totalX += x[1] - x[0]
        totalY += y[1] - y[0]
    print('Average ROI width: ', totalX/len(img), '\nAverage ROI length: ', totalY/len(img))
    
# View crops / save to disk
def writeCropsToDisk(img):
    count = 0
    for key in img:
        count+=1
        f = open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_crop/' + key + '.png', 'wb')
        w = png.Writer(width = CROP_SIZE, height = CROP_SIZE, bitdepth=16, greyscale=True)
        w.write(f, img[key]['crop'])
        f.close()
        print(count, '/', len(img))
        if count == -1:
            break
            
def buildArrayForPickle(img):    #img.update({key:{}})
    allCrops = {}
    for key in img:
        allCrops.update({key:[]})
        allCrops[key] = img[key]['crop']
    return allCrops
        
def savePickle(ob):
    import pickle
    print('Pickling...')
    with open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_malignantCrop256.pickle', 'wb') as output:
        pickle.dump(ob, output, pickle.HIGHEST_PROTOCOL)
        
def buildDictNormals(dicomImg, fileList):
    from scipy import ndimage
    # Get pixel values
    # Draw square around breast
    # Select random centre for crop within breast
    img = {}
    for i in range(len(fileList)):
        key = os.path.basename(fileList[i])[:-4]
        img.update({key:{}})
        img[key].update({'img': dicomImg[i].pixel_array})
        # Get centre of mass (breast centre)
        centre = ndimage.measurements.center_of_mass(img[key]['img'])
        centre = np.asarray(centre).astype(int)
        
#         #Check images
#         plt.figure(figsize=(20,20))  
#         #plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
#         plt.imshow(img[key]['img'], cmap='gray')
#         plt.plot(centre[1], centre[0], marker='x', color=[1,0,1], markersize=30)
#         #plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_full.png')
#         plt.show()
#         plt.close()

        #LATER
#         img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
#         img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})



def main():
    dicomImg, fileList = getFiles()
#     dicomImg, fileList = deletePreProcessed(dicomImg, fileList)
    #buildDictNormals(dicomImg, fileList)
    buildDict(dicomImg, fileList)
    img = buildDict(dicomImg, fileList)
    img = computeCrops(img)
#     findAverageROISize(img)
    findBitDepth(img)
#     writeMarkedImages(img)
#     writeCropsToDisk(img)
    savePickle(buildArrayForPickle(img)) 


if __name__ == "__main__":
    main()

print('Done')

Started...
1488  dicom images found
1488  Files found
Loading images...
1 / 1488
2 / 1488
3 / 1488
4 / 1488
5 / 1488
6 / 1488
7 / 1488
8 / 1488
9 / 1488
10 / 1488
11 / 1488
12 / 1488
13 / 1488
14 / 1488
15 / 1488
16 / 1488
17 / 1488
18 / 1488
19 / 1488
20 / 1488
21 / 1488
22 / 1488
23 / 1488
24 / 1488
25 / 1488
26 / 1488
27 / 1488
28 / 1488
29 / 1488
30 / 1488
31 / 1488
32 / 1488
33 / 1488
34 / 1488
35 / 1488
36 / 1488
37 / 1488
38 / 1488
39 / 1488
40 / 1488
41 / 1488
42 / 1488
43 / 1488
44 / 1488
45 / 1488
46 / 1488
47 / 1488
48 / 1488
49 / 1488
50 / 1488
51 / 1488
52 / 1488
53 / 1488
54 / 1488
55 / 1488
56 / 1488
57 / 1488
58 / 1488
59 / 1488
60 / 1488
61 / 1488
62 / 1488
63 / 1488
64 / 1488
65 / 1488
66 / 1488
67 / 1488
68 / 1488
69 / 1488
70 / 1488
71 / 1488
72 / 1488
73 / 1488
74 / 1488
75 / 1488
76 / 1488
77 / 1488
78 / 1488
79 / 1488
80 / 1488
81 / 1488
82 / 1488
83 / 1488
84 / 1488
85 / 1488
86 / 1488
87 / 1488
88 / 1488
89 / 1488
90 / 1488
91 / 1488
92 / 1488
93 / 1488
94 / 14

753 / 1488
754 / 1488
755 / 1488
756 / 1488
757 / 1488
758 / 1488
759 / 1488
760 / 1488
761 / 1488
762 / 1488
763 / 1488
764 / 1488
765 / 1488
766 / 1488
767 / 1488
768 / 1488
769 / 1488
770 / 1488
771 / 1488
772 / 1488
773 / 1488
774 / 1488
775 / 1488
776 / 1488
777 / 1488
778 / 1488
779 / 1488
780 / 1488
781 / 1488
782 / 1488
783 / 1488
784 / 1488
785 / 1488
786 / 1488
787 / 1488
788 / 1488
789 / 1488
790 / 1488
791 / 1488
792 / 1488
793 / 1488
794 / 1488
795 / 1488
796 / 1488
797 / 1488
798 / 1488
799 / 1488
800 / 1488
801 / 1488
802 / 1488
803 / 1488
804 / 1488
805 / 1488
806 / 1488
807 / 1488
808 / 1488
809 / 1488
810 / 1488
811 / 1488
812 / 1488
813 / 1488
814 / 1488
815 / 1488
816 / 1488
817 / 1488
818 / 1488
819 / 1488
820 / 1488
821 / 1488
822 / 1488
823 / 1488
824 / 1488
825 / 1488
826 / 1488
827 / 1488
828 / 1488
829 / 1488
830 / 1488
831 / 1488
832 / 1488
833 / 1488
834 / 1488
835 / 1488
836 / 1488
837 / 1488
838 / 1488
839 / 1488
840 / 1488
841 / 1488
842 / 1488
843 / 1488

1459 / 1488
1460 / 1488
1461 / 1488
1462 / 1488
1463 / 1488
1464 / 1488
1465 / 1488
1466 / 1488
1467 / 1488
1468 / 1488
1469 / 1488
1470 / 1488
1471 / 1488
1472 / 1488
1473 / 1488
1474 / 1488
1475 / 1488
1476 / 1488
1477 / 1488
1478 / 1488
1479 / 1488
1480 / 1488
1481 / 1488
1482 / 1488
1483 / 1488
1484 / 1488
1485 / 1488
1486 / 1488
1487 / 1488
1488 / 1488
MY_ERROR: key not found:
 1.2.840.113681.2229460924.948.3513224549.627
1487 DICOM images extracted
MY_ERROR: key not found:
 1.2.840.113681.2229460924.948.3513224549.627
1487 DICOM images extracted
Find bit depth...
16383
16383
16383
16383
6378
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
5292
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
2453
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
1

16383
16383
16383
16383
16383
16383
16383
16383
16383
5002
16383
4680
16383
16383
16383
16383
16383
15884
7701
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
11649
16383
16383
16383
16383
8105
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
4419
16383
16383
16383
16383
16383
16383
14603
16383
16383
5562
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
6952
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
10202
16383
16383
16383
16383
9445
13049
16383
16383
16383
16383
16383
16383
16383
16383
13952
16383
15426
16383
16383
16383
16383
16383
3540
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
14645
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
4823
16383
16383
16383
16383
5447
7089
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
16383
1638

In [8]:
print('hello world')
import pickle

hello world


In [41]:
# Read the pickle file
import pickle
objects = []
with (open("/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_malignantCrop256.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
# for key in image:
#     print(key)
#print(objects[0]['crop'].shape)
images = objects[0]

print('Find bit depth...')
maxmax = 0
for key in images:
    tmp = images[key]
    print(np.amax(tmp))
    if np.amax(tmp) > maxmax:
        maxmax = np.amax(tmp)
print('The largest value is: ', maxmax)




Find bit depth...
858
724
735
566
760
13717
657
618
511
760
2129
725
662
674
483
11723
772
566
1267
897
684
751
983
679
654
1556
861
565
729
673
679
788
616
7089
555
502
822
705
661
758
632
749
650
490
677
11926
11483
2129
704
668
665
1004
770
603
1051
682
1236
584
665
1050
654
830
1073
781
773
661
734
735
1319
724
797
729
665
998
744
781
9982
552
930
777
697
679
14222
773
763
952
631
773
658
791
532
747
1071
720
807
717
657
688
741
16383
729
780
617
772
536
522
533
618
674
982
607
798
798
694
694
742
731
1135
995
700
858
836
575
707
894
680
1075
722
1269
1115
668
732
753
563
865
1941
1400
661
768
772
1133
1154
697
509
615
16383
698
705
8948
616
867
864
2951
665
701
713
788
4739
875
684
884
7484
522
1137
1102
541
1113
740
617
1413
597
613
860
942
545
637
511
656
722
587
674
726
5446
685
816
1954
741
769
654
608
828
760
794
15812
2071
680
833
624
1211
704
572
604
606
582
769
599
718
535
577
594
905
502
569
504
749
655
584
785
16383
555
790
659
671
817
648
10970
763
752
12259
909
6188
75